# Solving TSP using Gurobi with lazy constraint

In [2]:
import time
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def read_cost_matrix(file_path):
    """Reads the cost matrix from the input file."""
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Locate the start of the EDGE_WEIGHT_SECTION
    start_index = lines.index("EDGE_WEIGHT_SECTION\n") + 1

    # Read the dimension
    dimension_line = next(line for line in lines if line.startswith("DIMENSION"))
    dimension = int(dimension_line.split(":")[-1].strip())

    # Parse the cost matrix
    cost_matrix = []
    for line in lines[start_index:]:
        if "EOF" in line:
            break
        cost_matrix.extend(map(int, line.split()))

    cost_matrix = np.array(cost_matrix).reshape((dimension, dimension))
    return cost_matrix

def solve_atsp(cost_matrix):
    """Solves the Asymmetric Traveling Salesman Problem using Gurobi."""
    n = cost_matrix.shape[0]

    # Create the model
    model = gp.Model("ATSP")

    # Decision variables: x[i, j] = 1 if we travel from city i to city j
    x = model.addVars(n, n, vtype=GRB.BINARY, name="x")

    # Objective: minimize the total travel cost
    model.setObjective(gp.quicksum(cost_matrix[i, j] * x[i, j] for i in range(n) for j in range(n)), GRB.MINIMIZE)

    # Constraints: each city is visited exactly once
    model.addConstrs(gp.quicksum(x[i, j] for j in range(n) if i != j) == 1 for i in range(n))
    model.addConstrs(gp.quicksum(x[i, j] for i in range(n) if i != j) == 1 for j in range(n))

    # Subtour elimination constraints (lazy constraints)
    def subtour_elimination(model, where):
        if where == GRB.Callback.MIPSOL:
            # Get the solution
            selected = model.cbGetSolution(x)

            # Find subtours in the solution
            tours = find_subtours(selected, n)
            if len(tours) > 1:
                for tour in tours:
                    model.cbLazy(gp.quicksum(x[i, j] for i in tour for j in tour if i != j) <= len(tour) - 1)

    model.Params.LazyConstraints = 1
    model.optimize(subtour_elimination)

    # Retrieve the solution
    if model.status == GRB.OPTIMAL:
        solution = model.getAttr("x", x)
        route = extract_route(solution, n)
        total_time = model.Runtime  # Get the runtime in seconds
        return route, model.objVal, total_time
    else:
        raise RuntimeError("No optimal solution found")

def find_subtours(selected, n):
    """Finds all subtours in the solution."""
    visited = [False] * n
    tours = []

    for i in range(n):
        if not visited[i]:
            tour = []
            current = i
            while not visited[current]:
                visited[current] = True
                tour.append(current)
                next_city = [j for j in range(n) if selected[current, j] > 0.5 and j != current]
                if next_city:
                    current = next_city[0]
            tours.append(tour)
    return tours

def extract_route(solution, n):
    """Extracts the route from the solution."""
    route = []
    current_city = 0
    while len(route) < n:
        route.append(current_city)
        next_city = [j for j in range(n) if solution[current_city, j] > 0.5 and j != current_city]
        if next_city:
            current_city = next_city[0]
    return route

if __name__ == "__main__":
    file_path = "instance.txt"
    cost_matrix = read_cost_matrix(file_path)
    route, total_cost, total_time = solve_atsp(cost_matrix)
    print(f"Optimal route: {' -> '.join(map(str, route))}")
    print('-'*100)
    print(f"Time taken to solve (seconds): {total_time}")
    print(f"Total travel cost: {total_cost}")


Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 886 rows, 196249 columns and 391612 nonzeros
Model fingerprint: 0x0fe5860d
Variable types: 0 continuous, 196249 integer (196249 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.50s
Presolved: 886 rows, 196249 columns, 391612 nonzeros
Variable types: 0 continuous, 196249 integer (196249 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      0.0000000e+00      2s
       1       1012   3.2400000e+08   1.3696625e-01      2s
       2       2406

# Solving TSP with Google OR tool and hybrid approch (OR Tool + Gurobi)

In [3]:
def solve_with_or_tools(cost_matrix):
    """Solve TSP using OR-Tools and return the route."""
    data = {"distance_matrix": cost_matrix, "num_vehicles": 1, "depot": 0}
    manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]), data["num_vehicles"], data["depot"])
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        route = []
        index = routing.Start(0)
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
        return route, solution.ObjectiveValue()
    else:
        raise RuntimeError("No solution found with OR-Tools")

def solve_atsp_with_gurobi(cost_matrix, warm_start_route):
    """Solves the ATSP using Gurobi, utilizing the warm start solution."""
    n = cost_matrix.shape[0]
    model = gp.Model("ATSP")

    x = model.addVars(n, n, vtype=GRB.BINARY, name="x")
    model.setObjective(gp.quicksum(cost_matrix[i, j] * x[i, j] for i in range(n) for j in range(n)), GRB.MINIMIZE)

    model.addConstrs(gp.quicksum(x[i, j] for j in range(n) if i != j) == 1 for i in range(n))
    model.addConstrs(gp.quicksum(x[i, j] for i in range(n) if i != j) == 1 for j in range(n))

    for i in range(len(warm_start_route) - 1):
        from_city = warm_start_route[i]
        to_city = warm_start_route[i + 1]
        x[from_city, to_city].start = 1

    model.Params.LazyConstraints = 1
    model.optimize()

    if model.status == GRB.OPTIMAL:
        solution = model.getAttr("x", x)
        route = extract_route(solution, n)
        total_time = model.Runtime
        return route, model.objVal, total_time
    else:
        raise RuntimeError("No optimal solution found")



if __name__ == "__main__":
    file_path = "instance.txt"
    cost_matrix = read_cost_matrix(file_path)

    print("Solving with OR-Tools...")
    start_or_tools = time.time()
    warm_start_route, or_tools_cost = solve_with_or_tools(cost_matrix)
    end_or_tools = time.time()
    or_tools_time = end_or_tools - start_or_tools

    print(f"OR-Tools Route: {' -> '.join(map(str, warm_start_route))}")
    print('-'*100)
    print(f"OR-Tools Cost: {or_tools_cost}")
    print(f"Time taken by OR-Tools: {or_tools_time:.2f} seconds")
    
    print('-'*100)
    print("\nSolving with Gurobi using OR-Tools warm start...")
    start_gurobi = time.time()
    route, total_cost, gurobi_time = solve_atsp_with_gurobi(cost_matrix, warm_start_route)
    end_gurobi = time.time()
    gurobi_total_time = end_gurobi - start_gurobi

    print(f"Gurobi Route: {' -> '.join(map(str, route))}")
    print('-'*100)
    print(f"Gurobi Cost: {total_cost}")
    print(f"Time taken by Gurobi solver: {gurobi_time:.2f} seconds")
    print(f"Total time for Gurobi (including overhead): {gurobi_total_time:.2f} seconds")


Solving with OR-Tools...
OR-Tools Route: 0 -> 269 -> 166 -> 87 -> 427 -> 423 -> 422 -> 442 -> 392 -> 388 -> 441 -> 437 -> 436 -> 434 -> 414 -> 402 -> 433 -> 428 -> 426 -> 432 -> 409 -> 404 -> 311 -> 415 -> 431 -> 413 -> 395 -> 430 -> 418 -> 412 -> 429 -> 425 -> 424 -> 420 -> 111 -> 194 -> 24 -> 301 -> 242 -> 231 -> 155 -> 108 -> 165 -> 419 -> 277 -> 196 -> 101 -> 224 -> 227 -> 331 -> 195 -> 176 -> 69 -> 190 -> 104 -> 46 -> 284 -> 247 -> 233 -> 173 -> 107 -> 114 -> 184 -> 254 -> 93 -> 105 -> 323 -> 362 -> 379 -> 361 -> 278 -> 271 -> 360 -> 106 -> 342 -> 283 -> 302 -> 356 -> 186 -> 171 -> 160 -> 84 -> 55 -> 349 -> 332 -> 213 -> 294 -> 115 -> 210 -> 91 -> 193 -> 45 -> 28 -> 6 -> 65 -> 256 -> 159 -> 335 -> 312 -> 371 -> 347 -> 264 -> 255 -> 346 -> 318 -> 316 -> 174 -> 267 -> 344 -> 41 -> 345 -> 62 -> 188 -> 74 -> 26 -> 187 -> 83 -> 51 -> 343 -> 244 -> 232 -> 36 -> 372 -> 52 -> 341 -> 38 -> 370 -> 200 -> 49 -> 151 -> 85 -> 306 -> 340 -> 211 -> 48 -> 272 -> 339 -> 265 -> 25 -> 336 -> 229 -> 

# Solution
- **Gurobi (lazy constraint)**:
  - Total travel cost: 2720.0
    
  - Time taken to solve (seconds): 11.56 seconds
    
- **Google OR Tools**:
  - OR-Tools Cost: 2763
    
  - Time taken by OR-Tools: 13.83 seconds
    
- **Hybrid approch (Google OR tool and Gurobi)**:
  - Gurobi Cost: 2720.0
    
  - Time taken by Gurobi solver: 2.70 seconds
    
  - Total time for Gurobi (including overhead): 17.55 seconds): 17.55 seconds55 secondsds  